In [1]:
library(SBC);
library(cmdstanr);
library(jsonlite);
library(tidyverse);
options(mc.cores = parallel::detectCores());
library(future);
plan(multisession);

options(SBC.min_chunk_size = 5);

cache_dir <- "./SBC_cache"
if(!dir.exists(cache_dir)) {
    dir.create(cache_dir)
}

ALPHA <- 0.05
model_BT_1 <- cmdstanr::cmdstan_model("../models/BT_model_1.stan")
model_BT_2 <- cmdstanr::cmdstan_model("../models/BT_model_2.stan")


This is cmdstanr version 0.7.1

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- CmdStan path: /Users/igor.michels/.cmdstan/cmdstan-2.34.1

- CmdStan version: 2.34.1


A newer version of CmdStan is available. See ?install_cmdstan() to install it.
To disable this check set option or environment variable CMDSTANR_NO_VER_CHECK=TRUE.

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data_generator_single_BT_1 <- function(n_clubs){
    n_seasons <- 1
    clubs <- sprintf("Club %02d", 1:n_clubs)
    clubs <- 1:n_clubs
    force <- rnorm(length(clubs))
    df <- data.frame(Club = clubs, Force = force)
    data <- merge(df, df, by = NULL) %>% filter(Club.x != Club.y)
    data$prob.home <- exp(data$Force.x) / (exp(data$Force.x) + exp(data$Force.y))
    data$home.wins <- 0

    for (i in 1:n_seasons) {
        data$random <- runif(n_clubs * (n_clubs - 1))
        data$home.wins <- data$home.wins + (data$random <  data$prob.home) * 1
    }

    data$away.wins <- n_seasons - data$home.wins
    data <- subset(data, select = -random)
    data <- subset(data, select = -prob.home)
    names(data) <- c("home_name", "home_force", "away_name", "away_force",
                     "home_wins", "away_wins")

    list(
        variables = list(
            skill = force
        ),
        generated = list(
            num_games = nrow(data),
            num_teams = n_clubs,
            team1 = data$home_name,
            team2 = data$away_name,
            team1_win = data$home_wins
        )
    )
}

In [3]:
data_generator_single_BT_2 <- function(n_clubs){
    n_seasons <- 1
    clubs <- sprintf("Club %02d", 1:n_clubs)
    clubs <- 1:n_clubs
    force <- rnorm(length(clubs))
    home_force <- rnorm(1)
    df <- data.frame(Club = clubs, Force = force)
    data <- merge(df, df, by = NULL) %>% filter(Club.x != Club.y)
    data$prob.home <- exp(data$Force.x + home_force) / (exp(data$Force.x) + exp(data$Force.y))
    data$home.wins <- 0

    for (i in 1:n_seasons) {
        data$random <- runif(n_clubs * (n_clubs - 1))
        data$home.wins <- data$home.wins + (data$random <  data$prob.home) * 1
    }

    data$away.wins <- n_seasons - data$home.wins
    data <- subset(data, select = -random)
    data <- subset(data, select = -prob.home)
    names(data) <- c("home_name", "home_force", "away_name", "away_force",
                     "home_wins", "away_wins")

    list(
        variables = list(
            skill = force,
            home_advantage = home_force
        ),
        generated = list(
            num_games = nrow(data),
            num_teams = n_clubs,
            team1 = data$home_name,
            team2 = data$away_name,
            team1_win = data$home_wins
        )
    )
}

In [4]:
set.seed(0)
n_sims <- 250
data_generator_BT_1 <- SBC_generator_function(data_generator_single_BT_1, n_clubs = 20)
dataset_BT_1 <- generate_datasets(data_generator_BT_1, n_sims)
backend_BT_1 <- SBC_backend_cmdstan_sample(model_BT_1, iter_warmup = 2000,
                                           iter_sampling = 2500, chains = 4)

results_BT_1 <- compute_SBC(dataset_BT_1, backend_BT_1,
                            cache_mode = "results",
                            cache_location = file.path(cache_dir, "results_BT_1"))
    
write.csv(results_BT_1$stats,
          file = "SBC_cache/results_BT_1.csv")

Cache file exists but the backend hash differs. Will recompute.



In [ ]:
set.seed(0)
n_sims <- 250
data_generator_BT_2 <- SBC_generator_function(data_generator_single_BT_2, n_clubs = 20)
dataset_BT_2 <- generate_datasets(data_generator_BT_2, n_sims)
backend_BT_2 <- SBC_backend_cmdstan_sample(model_BT_2, iter_warmup = 2000,
                                           iter_sampling = 2500, chains = 4)

results_BT_2 <- compute_SBC(dataset_BT_2, backend_BT_2,
                            cache_mode = "results",
                            cache_location = file.path(cache_dir, "results_BT_2"))
    
write.csv(results_BT_2$stats,
          file = "SBC_cache/results_BT_2.csv")

In [ ]:
plot_rank_hist(results_BT_1)
plot_ecdf(results_BT_1)
plot_ecdf_diff(results_BT_1)

In [ ]:
plot_rank_hist(results_BT_2)
plot_ecdf(results_BT_2)
plot_ecdf_diff(results_BT_2)

In [ ]:
results <- results_BT_1
graph <- plot_ecdf(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df1 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df1$out <- (df1$ymax < df1$y) + (df1$ymin > df1$y)
df1 <- df1 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df1$out_ratio <- df1$out_ratio / length(unique(ecdf$x))
df1$out <- df1$out_ratio > ALPHA

graph <- plot_ecdf_diff(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df2 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df2$out <- (df2$ymax < df2$y) + (df2$ymin > df2$y)
df2 <- df2 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df2$out_ratio <- df2$out_ratio / length(unique(ecdf$x))
df2$out <- df2$out_ratio > ALPHA

final_df <- merge(df1, df2, by = "PANEL", suffixes = c("", "_diff"))
c(mean(as.numeric(final_df$out)), mean(as.numeric(final_df$out_diff)))

In [ ]:
results <- results_BT_2
graph <- plot_ecdf(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df1 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df1$out <- (df1$ymax < df1$y) + (df1$ymin > df1$y)
df1 <- df1 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df1$out_ratio <- df1$out_ratio / length(unique(ecdf$x))
df1$out <- df1$out_ratio > ALPHA

graph <- plot_ecdf_diff(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df2 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df2$out <- (df2$ymax < df2$y) + (df2$ymin > df2$y)
df2 <- df2 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df2$out_ratio <- df2$out_ratio / length(unique(ecdf$x))
df2$out <- df2$out_ratio > ALPHA

final_df <- merge(df1, df2, by = "PANEL", suffixes = c("", "_diff"))
c(mean(as.numeric(final_df$out)), mean(as.numeric(final_df$out_diff)))

In [ ]:
data <- read_json('../real_data/BT_model_data.json')

fit_BT_1 <- model_BT_1$sample(
  data = data,
  chains = 4,
  parallel_chains = 4,
  refresh = 500
)

In [ ]:
fit_BT_1$summary(
  variables = NULL,
  posterior::default_summary_measures(),
  extra_quantiles = ~posterior::quantile2(., probs = c(.0275, .975))
)

In [ ]:
data <- read_json('../real_data/BT_model_data.json')

fit_BT_2 <- model_BT_2$sample(
  data = data,
  chains = 4,
  parallel_chains = 4,
  refresh = 500
)

In [ ]:
fit_BT_2$summary(
  variables = NULL,
  posterior::default_summary_measures(),
  extra_quantiles = ~posterior::quantile2(., probs = c(.0275, .975))
)